<h1>Explorando el Interior de Modelos de Lenguaje Transformer</h1>
<i>Un análisis profundo de la arquitectura transformer en modelos generativos de lenguaje</i>

---

Este cuaderno está adaptado del Capítulo 3 del libro [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961)

### [OPCIONAL] - Instalación de Paquetes en <img src="https://colab.google/static/images/icons/colab.png" width=100>

Si estás ejecutando este cuaderno en Google Colab, necesitas **descomentar y ejecutar** el siguiente bloque de código para instalar las dependencias:

---

💡 **NOTA**: Para mejores resultados, usa una GPU. En Google Colab, ve a:
**Entorno de ejecución > Cambiar tipo de entorno de ejecución > Acelerador de hardware > GPU > Tipo de GPU > T4**.

---

In [1]:
%%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0

# Cargando el Modelo de Lenguaje

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Cargar modelo y tokenizador
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

# Crear un pipeline de generación de texto
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

# Entradas y Salidas de un Modelo Transformer Entrenado

In [ ]:
# Ejemplo adaptado al contexto argentino
prompt = "Escribe un mensaje disculpándote con Juan por haber pisado su pie en el colectivo. Explica cómo sucedió."

output = generator(prompt)

print(output[0]['generated_text'])

In [ ]:
# Mostrar la arquitectura del modelo
print(model)

# Seleccionando un token de la distribución de probabilidades (sampling/decoding)

In [ ]:
prompt = "La capital de Argentina es"

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

# Obtener la salida del modelo antes de la capa lm_head
model_output = model.model(input_ids)

# Obtener la salida de la capa lm_head
lm_head_output = model.lm_head(model_output[0])

In [ ]:
# Obtener el token con mayor probabilidad
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

# Acelerando la generación con caché de keys y values

In [ ]:
prompt = "Escribe una carta muy larga explicando por qué el asado del domingo se quemó. Describe qué salió mal."

# Tokenizar el prompt de entrada
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [ ]:
%%timeit -n 1
# Generar texto con caché activado
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=True
)

In [ ]:
%%timeit -n 1
# Generar texto con caché desactivado
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=False
)

# Explicación

## ¿Qué aprendimos?

1. **Carga de modelos**: Cargamos el modelo Phi-3-mini, un modelo de lenguaje eficiente y potente.
2. **Generación de texto**: Vimos cómo el modelo puede generar texto coherente en español con ejemplos locales.
3. **Arquitectura interna**: Exploramos la estructura del modelo transformer.
4. **Proceso de decodificación**: Analizamos cómo el modelo selecciona la siguiente palabra.
5. **Optimización**: Comparamos el rendimiento con y sin caché de atención.

## Aplicaciones

Estos modelos pueden usarse para:
- Generar contenido educativo en español rioplatense
- Analizar textos legales y normativas locales
- Crear asistentes virtuales con conocimiento de cultura argentina
- Procesar documentos históricos y literarios nacionales